In [10]:
import os
os.environ['PYTENSOR_FLAGS'] = 'mode=FAST_COMPILE,optimizer=fast_compile,floatX=float32,cxx='

import numpy as np
import pymc as pm
import pytensor.tensor as pt

In this notebook, we demonstrate the initialization capability of PyMC's ADVI.

We will fit a simple 1d normal distribution using ADVI and compare the initializations.


In [ ]:
# run PyMC ADVI algorithm
with pm.Model() as init_model:
    # Define priors
    mu = pm.Normal("mu", 0, 1)
    likelihood = pm.Normal('y', mu=mu, sigma=1, observed=None)
    init_fit = pm.fit(0, method=pm.ADVI(), start={'mu': 1.0, 'sigma': 1.0})

# We try initializing using advi.fit instead of pm.fit since this is the other method of pymc ADVI initialization
with pm.Model() as init_pm_model:
    # Define priors
    mu = pm.Normal("mu", 0, 1)
    likelihood = pm.Normal('y', mu=mu, sigma=1, observed=None)

    advi = pm.ADVI()
    try:
        advi_fit = advi.fit(0, start={'mu': 1.0, 'sigma': 1.0})
    except Exception as e:
        print(f"ADVI fit does not support start function: {e}")

with pm.Model() as default_model:
    # Define priors
    mu = pm.Normal("mu", 0, 1)
    likelihood = pm.Normal('y', mu=mu, sigma=1, observed=None)
    default_fit = pm.fit(0, method=pm.ADVI())



Output()

Initialization only


Output()

ADVI fit does not support start function


Initialization only


In [12]:
# we figure out which index mu is - just to make sure we are looking at the correct value
default_mu_idx = [var.name for var in default_model.unobserved_RVs].index("mu")
init_mu_idx = [var.name for var in init_model.unobserved_RVs].index("mu")
default_mu_idx, init_mu_idx

(0, 0)

In [13]:
# This is what the default initialized to
default_fit.mean.eval()[default_mu_idx], default_fit.std.eval()[default_mu_idx]

(np.float32(0.0), np.float32(0.6931472))

In [14]:
# This is what the init initialized to, 
# when we told it to use start={'mu': 1.0, 'sigma': 1.0}
init_fit.mean.eval()[init_mu_idx], init_fit.std.eval()[init_mu_idx]

(np.float32(0.0), np.float32(0.6931472))